In [ ]:
#Datasets to pre-train the model
# dataset https://huggingface.co/datasets/katielink/moleculenet-benchmark
# https://huggingface.co/datasets/sagawa/pubchem-10m-canonicalized
#Most lickly to fine tune: https://huggingface.co/GT4SD/multitask-text-and-chemistry-t5-base-augm
#https://github.com/GT4SD/multitask_text_and_chemistry_t5
#https://github.com/GLambard/Molecules_Dataset_Collection/blob/master/latest/HIV.csv
#https://colab.research.google.com/github/huggingface/blog/blob/main/notebooks/01_how_to_train.ipynb#scrollTo=IMnymRDLe0hi

In [ ]:
!pip install datasets transformers

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [ ]:
import transformers

print(transformers.__version__)

4.35.0


In [ ]:
from datasets import load_dataset
data_pub = load_dataset('sagawa/pubchem-10m-canonicalized', 'data')

In [ ]:
#local download
!git clone https://huggingface.co/datasets/katielink/moleculenet-benchmark

Cloning into 'moleculenet-benchmark'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 59 (delta 5), reused 0 (delta 0), pack-reused 4
Unpacking objects: 100% (59/59), 4.54 MiB | 4.89 MiB/s, done.
Filtering content: 100% (2/2), 51.54 MiB | 24.68 MiB/s, done.


In [ ]:
from datasets import load_dataset
data_files = {"train": "train.csv", "test": "test.csv"}
datasets = load_dataset('/content/moleculenet-benchmark/hiv', data_files=data_files)

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,smiles,activity,HIV_active
0,COc1ccc(N2C(=O)C(=Cc3cccc(Oc4ccccc4)c3)SC2c2ccccc2)cc1,CI,0
1,CC(C)=CCCC(C)=CCCC1(C)OC(=O)C2CC21,CI,0
2,COCCCC(=O)NC(c1ccccc1)(c1ccccc1)c1ccccc1,CI,0
3,O=c1cc(O)c2c(O)cc(O)cc2o1,CI,0
4,O=S(=O)(c1ccccc1)C1(Sc2ccccc2)CC(Br)C(Br)C1,CI,0
5,Nc1ccc(C=Cc2ccc3ncccc3c2)cc1,CI,0
6,O=C(CC(=O)N(C(=O)C=Cc1ccccc1)c1ccc(Cl)cc1)N1N=C(c2ccccc2)C(N=Nc2cccc([N+](=O)[O-])c2)C1=O,CI,0
7,CCc1c(O)nc2ccc(N)cc2c1C,CI,0
8,Cc1c(OCCCOc2ccc3c(=O)cc(-c4ccccc4)oc3c2C)ccc2c(=O)cc(-c3ccccc3)oc12,CI,0
9,N=c1[nH]c2cc(Cl)ccc2n1N,CI,0


In [ ]:

vocab_path = '/content/smiles_vocab.txt'

In [ ]:
smiles = datasets["train"]["smiles"]

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd

df = pd.DataFrame(datasets["train"]["smiles"])
smile = df.to_csv('/content/smiles_only/smilecsv.csv')

,0
0,C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...
1,CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21
2,Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1
3,O=S(=O)(O)CCS(=O)(=O)O
4,CCOP(=O)(Nc1cccc(Cl)c1)OCC
...,...
32895,Cc1cccc(N2C(=O)C3c4[nH]c5ccc(C)cc5c4C4CCC(C(C)...
32896,CCC1CCC2c3c([nH]c4ccc(C)cc34)C3C(=O)N(N(C)C)C(...
32897,Cc1ccc2[nH]c3c(c2c1)C1CCC(C(C)(C)C)CC1C1C(=O)N...
32898,Cc1ccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C...


In [ ]:
from transformers  import BertGenerationTokenizer #can try the BERT generation

from tokenizers import ByteLevelBPETokenizer


In [ ]:
%%time
tokenizer = ByteLevelBPETokenizer()
tokenizer.train('/content/smiles_only/smilecsv.csv', vocab_size=len(datasets), min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])


CPU times: user 3.37 s, sys: 28 ms, total: 3.4 s
Wall time: 2.59 s


In [ ]:
!mkdir test_vocab_new
tokenizer.save_model("test_vocab_new")

['test_vocab_new/vocab.json', 'test_vocab_new/merges.txt']

In [ ]:
!pip install sentencepiece

In [ ]:
#Check later

import transformers
from transformers  import BertGenerationTokenizer, BertGenerationEncoder, BertGenerationDecoder
tokenizer = BertGenerationTokenizer('/content/smiles_only/smilecsv.csv')

RuntimeError: ignored

In [ ]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing


